# 🎬 A.V.E.A - Cloud Edition

Run your own AI Video Editor in the cloud. No timeout issues, super fast rendering.

### Instructions:
1.  Run the **Setup** cell to install everything.
2.  Enter your **Google API Key** when prompted.
3.  Run the **Upload & Process** cell to select your files and generate the video.

In [ ]:
# @title 1. Setup Environment
import os
import sys
import subprocess

def run_console_command(cmd):
    """Runs a shell command in a way compatible with Jupyter and terminals."""
    if 'IPython' in sys.modules:
        from IPython import get_ipython
        if get_ipython():
            get_ipython().system(cmd)
            return
    # Fallback
    subprocess.run(cmd, shell=True, check=False)

print("⚙️ Setting up environment...")

# 1. Install Python packages (Universal Method)
print("📦 Installing Python libraries...")
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "google-generativeai", "moviepy", "gTTS"])

# 2. Linux/Colab System Dependencies
if os.name == 'posix':
    print("🐧 Linux/Colab environment detected. Installing system dependencies...")
    run_console_command('apt-get update -qq')
    run_console_command('apt-get install imagemagick -qq')
    # Fix ImageMagick policy to allow text rendering
    run_console_command("sed -i 's/none/read,write/g' /etc/ImageMagick-6/policy.xml")
else:
    print("🪟 Windows/Local environment detected. Skipping 'apt-get'. Ensure ImageMagick is installed manually.")

print("✅ Environment Ready!")

In [ ]:
# @title 2. Core Logic (A.V.E.A Engine)
import os
import time
import json
import asyncio
import concurrent.futures
import traceback
from typing import List, Dict, Any
from datetime import datetime

import google.generativeai as genai
try:
    from google.colab import files, userdata
    IN_COLAB = True
except ImportError:
    IN_COLAB = False
    print("⚠️ Running locally. 'files.upload()' will use fallback.")

from moviepy.editor import (
    VideoFileClip, ImageClip, concatenate_videoclips, CompositeVideoClip, TextClip, AudioFileClip, CompositeAudioClip
)
from moviepy.config import change_settings
from gTTS import gTTS

# Configure ImageMagick Dynamic Path
if os.name == 'posix':
    change_settings({"IMAGEMAGICK_BINARY": "/usr/bin/convert"})
else:
    # Attempt to find common Windows paths or assume it's in PATH
    possible_paths = [
        r"C:\Program Files\ImageMagick-7.1.2-Q16-HDRI\magick.exe",
        r"C:\Program Files\ImageMagick-7.1.3-Q16-HDRI\magick.exe"
    ]
    found = False
    for p in possible_paths:
        if os.path.exists(p):
            change_settings({"IMAGEMAGICK_BINARY": p})
            print(f"Found ImageMagick at: {p}")
            found = True
            break
    if not found:
        print("Windows: ImageMagick binary not found in exact path. Assuming it's in system PATH.")

# --- 1. GEMINI CLIENT ---

def _upload_single_file(path: str):
    """Uploads a file to Gemini and waits for processing."""
    print(f"Uploading {os.path.basename(path)}...")
    file_ref = genai.upload_file(path=path)
    
    while file_ref.state.name == "PROCESSING":
        time.sleep(1)
        file_ref = genai.get_file(file_ref.name)
        
    if file_ref.state.name != "ACTIVE":
        print(f"WARNING: File {file_ref.name} failed processing.")
        return None
    return file_ref

async def analyze_media(media_paths, api_key, style="Motivational", duration=0, aspect_ratio="16:9"):
    if not api_key:
        raise ValueError("API Key is missing!")
    
    genai.configure(api_key=api_key)
    
    # Parallel Upload (Max 5 workers to be safe)
    print(f"\n🚀 Sending {len(media_paths)} files to Gemini...")
    uploaded_refs = []
    
    loop = asyncio.get_running_loop()
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        tasks = [
            loop.run_in_executor(executor, _upload_single_file, path)
            for path in media_paths
        ]
        results = await asyncio.gather(*tasks)
        
    uploaded_refs = [f for f in results if f is not None]
    
    # Construct Prompt
    duration_text = f"Target Length: {duration} sec."
    timing_rule = f"- Strictly end at {duration}s."
    
    if duration == 0:
        duration_text = "Target Length: Flexible (Auto-detect based on best content). Aim for 30-60s."
        timing_rule = "- FLEXIBLE TIMING: End when the story is complete."

    prompt = f"""
You are an expert video editor.
I have provided {len(uploaded_refs)} media files above.

Task: Create a viral {style} style video.
{duration_text}
Format: {aspect_ratio} ({'Horizontal/YouTube' if aspect_ratio == '16:9' else 'Vertical/Reel'}).

CRITICAL INSTRUCTIONS:
1. Visual Reasoning: Identify the most interesting moments.
2. Dynamic Cuts: Do NOT just use the start. Find the BEST segments.
3. Pacing: Use at least 4-6 scenes.
4. Story: Hook -> Body -> Punch.

IMPORTANT TIMING RULES:
{timing_rule}
- Start/End timestamps are in SECONDS.
- Minimum clip length: 2.0 seconds.

Output JSON Format (Strict):
{{
  "scenes": [
    {{
      "input_type": "user_clip" | "user_image",
      "file_path": "original_filename", 
      "start": 0.0,
      "end": 4.0, 
      "duration": 4.0,
      "role": "hook" | "body" | "punch",
      "caption": "Overlay text",
      "effect": "slow_zoom_in" | "none"
    }}
  ]
}}

IMPORTANT: Match 'file_path' to original filenames where possible.
original filenames: {json.dumps([os.path.basename(p) for p in media_paths])}
"""

    print("🧠 Analyzing content...")
    model = genai.GenerativeModel("gemini-1.5-flash")
    response = model.generate_content(uploaded_refs + [prompt])
    
    try:
        text = response.text.replace('```json', '').replace('```', '')
        storyboard = json.loads(text)
        storyboard['aspect_ratio'] = aspect_ratio
        return storyboard
    except Exception as e:
        print(f"JSON Parse Error: {e}")
        print("Raw Response:", response.text)
        return None

# --- 2. RENDERER ---

def render_video(storyboard, output_path, media_map):
    scenes = storyboard.get("scenes", [])
    aspect_ratio = storyboard.get("aspect_ratio", "9:16")
    target_w, target_h = (1920, 1080) if aspect_ratio == "16:9" else (1080, 1920)
    
    clips = []
    full_audio_clips = []
    
    print(f"\n🎥 Rendering {len(scenes)} scenes ({aspect_ratio})...")

    try:
        for i, scene in enumerate(scenes):
            fname = scene.get("file_path")
            # Find local path
            local_path = None
            for p in media_map:
                if os.path.basename(p) == fname:
                    local_path = p
                    break
            # Fallback if AI messed up name
            if not local_path and i < len(media_map):
                local_path = list(media_map.values())[i % len(media_map)]

            if not local_path or not os.path.exists(local_path):
                print(f"Skipping missing file: {fname}")
                continue

            input_type = scene.get("input_type")
            start, end = float(scene.get("start", 0)), float(scene.get("end", 5))
            duration = float(scene.get("duration", end - start))
            caption = scene.get("caption", "")
            effect = scene.get("effect", "")

            # Create Clip
            clip = None
            if input_type == "user_clip":
                clip = VideoFileClip(local_path).subclip(start, start + duration)
            else:
                clip = ImageClip(local_path).set_duration(duration)

            # Resize/Crop
            clip = clip.resize(height=target_h)
            if clip.w > target_w:
                clip = clip.crop(x1=clip.w/2 - target_w/2, width=target_w)
            else:
                clip = clip.resize(width=target_w).crop(y1=clip.h/2 - target_h/2, height=target_h)

            # Effect
            if effect == "slow_zoom_in":
                clip = clip.fx(lambda c: c.resize(lambda t: 1 + 0.04 * t))

            # Caption
            final_clip = clip
            if caption:
                try:
                    txt = TextClip(caption, fontsize=70, color='white', font='Arial-Bold', method='caption', size=(target_w * 0.8, None), stroke_color='black', stroke_width=2)
                    txt = txt.set_pos(('center', 'bottom')).set_duration(clip.duration)
                    final_clip = CompositeVideoClip([clip, txt])
                except Exception as e:
                    print(f"Caption Error: {e}")

            # Voiceover (TTS)
            if caption:
                try:
                    tts = gTTS(text=caption, lang='en')
                    tts_path = f"tts_{i}.mp3"
                    tts.save(tts_path)
                    audio = AudioFileClip(tts_path)
                    full_audio_clips.append(audio.set_start(sum(c.duration for c in clips)))
                except Exception as e:
                    pass

            clips.append(final_clip)

        # Concatenate
        final_video = concatenate_videoclips(clips, method="compose")
        
        # Audio Mix
        if full_audio_clips:
            # Clamp audio to video length
            final_audio = CompositeAudioClip(full_audio_clips).set_duration(final_video.duration)
            # Mix with original audio if present
            if final_video.audio:
                final_video.audio = CompositeAudioClip([final_video.audio.volumex(0.3), final_audio])
            else:
                final_video.audio = final_audio

        final_video.write_videofile(output_path, fps=24, preset='ultrafast', logger=None)
        print(f"\n✅ Video Generated: {output_path}")
        return output_path

    except Exception as e:
        print(f"Render Error: {e}")
        traceback.print_exc()
        return None

In [ ]:
# @title 3. Run Generator

# 1. Upload Files
print("Upload your media files (Videos/Images):")
try:
    if IN_COLAB:
        uploaded = files.upload()
        media_paths = list(uploaded.keys())
    else:
        # Local fallback simulation
        print("Running locally? Please specify file paths here manually for testing, or use standard I/O.")
        # media_paths = ["file1.mp4", ...] 
        media_paths = []
except Exception as e:
    print(f"Upload Error: {e}")
    media_paths = []

media_map = {name: name for name in media_paths}

# 2. Settings
API_KEY = input("Enter Google API Key: ")
STYLE = "Cinematic" #@param ["Cinematic", "Motivational", "Vlog", "Fast-Paced"]
ASPECT_RATIO = "16:9" #@param ["9:16", "16:9"]
DURATION = 0 #@param {type:"integer"} 
# 0 = Auto

# 3. Execute
if media_paths:
    print("\n--- Starting A.V.E.A ---")
    # Note: In a real notebook, you'd run asyncio directly. 
    # Jupyter handles top-level awaits.
    storyboard = await analyze_media(media_paths, API_KEY, STYLE, DURATION, ASPECT_RATIO)
    
    if storyboard:
        print("\nStoryboard created! Scenes:", len(storyboard.get("scenes", [])))
        output_file = "final_video.mp4"
        result = render_video(storyboard, output_file, media_map)
        
        if result and IN_COLAB:
            files.download(result)
    else:
        print("Failed to create storyboard.")
else:
    print("No files uploaded.")